In [1]:
import pandas
import numpy as np
import cv2
import os

In [4]:
def kelas (file):
    if 'ha' in file: dot=1
    if 'na' in file: dot=2
    if 'ca' in file: dot=3
    if 'ra' in file: dot=4
    if 'ka' in file: dot=5
    if 'da' in file: dot=6
    if 'ta' in file: dot=7
    if 'sa' in file: dot=8
    if 'wa' in file: dot=9
    if 'la' in file: dot=10
    if 'ma' in file: dot=11
    if 'ga' in file: dot=12
    if 'ba' in file: dot=13
    if 'nga' in file: dot=14
    if 'pa' in file: dot=15
    if 'ja' in file: dot=16
    if 'ya' in file: dot=17
    if 'nya' in file: dot=18
    return (dot-1)

def minmax (img):#normalisasi min max
    x,y = img.shape
    balik = np.zeros(img.shape)
    maks  = max(img.flatten())
    mins = min(img.flatten())
    for i in range(x):
        for j in range (y):
            balik[i,j]=(img[i,j]-mins)/(maks-mins)
    return balik

def scale (img):
    return img/255

In [6]:
filepath = "D:\\data"

data = []
label =[]

for a in os.listdir(filepath):
    for b in os.listdir(filepath + "\\" + str(a)):
        img= cv2.imread(filepath + "\\" + str(a) + "\\" + str(b),0)#grayscale
        x = cv2.resize(img,(64,64))#resize
        x = scale(x)
        data.append(x)
        label.append(kelas (b))

In [8]:
data = np.array(data)
label = np.array(label)

from sklearn.model_selection import train_test_split

x1,x2,y1,y2= train_test_split(data,label,test_size=0.2)

In [34]:
dtrain = np.array(x1)
dlabel = np.array(y1)
ttrain = np.array(x2)
tlabel = np.array(y2)
print(tlabel.shape, dlabel.shape)

(2160,) (8640,)


In [27]:
fdtrain = np.reshape(dtrain,(8640,64,64,1))
fttrain = np.reshape(ttrain,(2160,64,64,1))

#fdtrain[:,:,:]= dtrain
print(fdtrain.shape, fttrain.shape)

(8640, 64, 64, 1) (2160, 64, 64, 1)


In [19]:
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [47]:
model = models.Sequential()
model.add(layers.Conv2D(128, (5, 5), activation='relu',padding='same', input_shape=(64, 64,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (5, 5),padding='same', activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(128, (5, 5),padding='same', activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(18, activation='softmax'))

In [48]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 128)       3328      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 128)       409728    
_________________________________________________________________
flatten_3 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                4194336   
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 18)               

In [58]:
import tensorflow as tf
from tensorflow import keras
import time

opt = keras.optimizers.Adam(learning_rate=0.2)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])#, tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
mulai = time.time()
history = model.fit(fdtrain,dlabel,
    steps_per_epoch = 240,      
    #batch_size=60,
    epochs = 10,
    validation_data = (fttrain,tlabel)
    )
print("waktu training adalah : ", time.time()-mulai , " s")

Train on 8640 samples, validate on 2160 samples
Epoch 1/10
8640/8640 [==============================] - 608s 70ms/sample - loss: 2.8905 - accuracy: 0.0561 - val_loss: 2.8919 - val_accuracy: 0.0477
Epoch 2/10
8640/8640 [==============================] - 526s 61ms/sample - loss: 2.8904 - accuracy: 0.0573 - val_loss: 2.8923 - val_accuracy: 0.0449
Epoch 3/10
8640/8640 [==============================] - 478s 55ms/sample - loss: 2.8904 - accuracy: 0.0566 - val_loss: 2.8924 - val_accuracy: 0.0449
Epoch 4/10
8640/8640 [==============================] - 483s 56ms/sample - loss: 2.8904 - accuracy: 0.0551 - val_loss: 2.8927 - val_accuracy: 0.0449
Epoch 5/10
8640/8640 [==============================] - 435s 50ms/sample - loss: 2.8904 - accuracy: 0.0582 - val_loss: 2.8927 - val_accuracy: 0.0449
Epoch 6/10
8640/8640 [==============================] - 451s 52ms/sample - loss: 2.8904 - accuracy: 0.0561 - val_loss: 2.8927 - val_accuracy: 0.0449
Epoch 7/10
8640/8640 [==============================] - 44